In [ ]:
!mkdir -p /tmp/pip/cache/
!cp ../input/resources-for-google-landmark-recognition-2020/efficientnet_pytorch-0.6.3-py3-none-any.whl /tmp/pip/cache/
!pip install --no-index --find-links /tmp/pip/cache/ efficientnet_pytorch

In [ ]:

from __future__ import print_function, division
import numpy as np
import pandas as pd
import torch

import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np
import torch.nn.functional as FUN
import os
from scipy import io
import json
from efficientnet_pytorch import EfficientNet
from PIL import Image, ImageDraw, ImageFont

OUTPUT_DIR = './'
input_size = 224
class_num = 5
image_dir = '../input/cassava-leaf-disease-classification/test_images/'
use_gpu = torch.cuda.is_available()
labellat=[]
filelist = os.listdir(image_dir)
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

def test_model(model):
    model.eval()
    tfms = transforms.Compose([transforms.Resize(300), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
    for m in range(len(filelist)):
        image = Image.open(image_dir+filelist[m])
        img = tfms(image).unsqueeze(0)
        img = Variable(img.cuda())

        labels_map = json.load(open('../input/5leijihe/5lei.txt'))
        labels_map = [labels_map[str(i)] for i in range(5)]


        with torch.no_grad():
            outputs = model(img)
        # Print predictions
        print('-----')
        cout = 0
        pp=0
        for idx in torch.topk(outputs, k=5).indices.squeeze(0).tolist():
            cout += 1
            prob = torch.softmax(outputs, dim=1)[0, idx].item()
            #print('{label:<75} ({p:.2f}%)'.format(label=labels_map[idx], p=prob*100))


            if pp<prob*100:
                pp=prob*100
                print('{label:<75} ({p:.2f}%)'.format(label=labels_map[idx], p=prob*100))
                labellat.append(idx)
    a=np.array(filelist)
    b=np.array(labellat)
    required_details = []
    for i in range(len(b)):
        required_details.append([a[i]])
        
        required_details.append([ b[i]])
    red=np.array(required_details)
    
    sub = pd.DataFrame(red.reshape(-1,2), columns=['image_id', 'label'])
    sub.to_csv(OUTPUT_DIR+'submission.csv', index=False)
    sub.head()
        



if __name__ == '__main__':
    model_ft = EfficientNet.from_name('efficientnet-b4')
    num_ftrs = model_ft._fc.in_features
    print(num_ftrs)
    model_ft._fc = nn.Linear(num_ftrs, class_num)
    
    
    if use_gpu:
        model_ft = model_ft.cuda()
    
    print('-' * 10)
    print('Test Accuracy:')
    model_ft.load_state_dict(torch.load("../input/darknesszx/tf_efficientnet_b4_ns_fold_0_9"))
    # criterion = nn.CrossEntropyLoss().cuda()
    test_model(model_ft)